Here is the code for the dstfp notebook so you can use the functions on your own systems!

In [ ]:
### Imports:

import numpy as np
import tellurium as te
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.optimize as opt
import sympy
from math import sqrt

1-Dimension Bifurcation Diagram (in two parts):

In [ ]:
def bifurcation_plot(f,r,x,tolerance,rlabel='r'):
    """ produce a bifurcation diagram for a function f(r,x) given
        f over a domain given by numpy arrays for parameter r and 
        state variable x
        
        f(r,x)  :  RHS function of autonomous ode dx/dt = f(r,x)
        r       :  numpy array giving r coordinates of domain
        x       :  numpy array giving x coordinates of domain
        rlabel  :  string for x axis parameter label
    """

    # set up a mesh grid and extract the 0 level set of f
    R,X = np.meshgrid(r,x)
    plt.figure()
    CS = plt.contour(R,X,f(R,X),[0],colors='k')
    plt.clf()

    # Compute the derivative with Sympy
    x1 = sympy.Symbol('x')
    r1 = sympy.Symbol('r')
    f_x = sympy.diff(f(r1,x1),x1)
    f_x = sympy.lambdify((r1,x1),f_x)
    
    c0 = CS.collections[0]
    # for each path in the contour extract vertices and mask by the sign of df/dx
    for path in c0.get_paths():
        vertices = path.vertices
        vr = vertices[:,0]
        vx = vertices[:,1]
        mask = f_x(vr,vx)
        stable = mask < 0.
        unstable = mask > 0.
        bif = abs(mask) < tolerance
        
        # plot the stable and unstable branches for each path

        plt.plot(vr[stable],vx[stable],marker='h',color='b',markersize=1,linestyle='None')
        plt.plot(vr[unstable],vx[unstable],marker='h',color='r',markersize=1,linestyle='None')
        plt.plot(vr[bif],vx[bif],marker='h',color='black',markersize=8,linestyle='None')
        
    
    plt.xlabel('parameter {0}'.format(rlabel))
    plt.ylabel('state variable x')
    plt.legend(('stable','unstable','bifurcation'),loc='best')
    plt.xlim(r[0],r[-1])
    plt.ylim(x[0],x[-1])
    plt.show()

In [ ]:
## First, define your function!
f = lambda h,x: -x**3 + 5*x + h
## Next, define the range of the parameter h and the range of state variable x
## note that the third value of the linspace functions is the number of points in the mesh, so values that are too
## low may miss bifurcation points. Conversely, values that are too high will take a long time to compute.
h = np.linspace(-7,7,10000)
x = np.linspace(-5,5,10000)
## Set your tolerance here. Tolerance can be adjusted in correspondence with the resolution of the plot below to ensure
### you are getting the correct approximations of the bifurcation points. 
tolerance = 0.01
## Finally, call the function to plot the bifurcation diagram
bifurcation_plot(f,h,x,tolerance,rlabel='c')

Flow in 2-Dimensions:

In [ ]:
x = np.linspace(-4,4,3000)
y = np.linspace(-4,4,3000)
X,Y = np.meshgrid(x, y)

U = X**3+Y**2-0.25
V = -Y**3+X
F1 = ((U)<0.0000000000001) & ((U)>-0.0000000000001).any()
F2 = ((V)<0.0000000000001) & ((V)>-0.0000000000001).any()
fig, ax = plt.subplots()
ax.set_aspect('equal', 'box')
ax.streamplot(X, Y, U, V, color='gray')
plt.xlabel("X")
plt.ylabel("Y")
plt.show()

Define ODEs:

In [ ]:
# DEFINE YOUR ODEs HERE:

## ODE for State Variable 1
def S1_ODE(X,Y):
    S1_M = X**3+Y**2-0.25
    return(S1_M)

## ODE for State Variable 2
def S2_ODE(X,Y):
    S2_M = -Y**3+X
    return(S2_M)

Nullclines in 2-dimensions:

In [ ]:
# CODE FOR NULLCLINE AND FLOW

## To use this function, the user needs to define both of their ODEs as a function of X and Y, with the names S1_ODE and S2_ODE,
## respectively. These ODEs then get used to generate a flow and nullclines in a specified range. Two important inputs are the 
## resolution and the tolerance, which need to be quite large and small respectively in order to get smooth lines and a 
## reliable plot. 

# FUNCTION FOR NULLCLINES AND FLOW
def gen_null_flow(min_S1, max_S1, min_S2, max_S2, resolution, tolerance):

    x = np.linspace(min_S1, max_S1, resolution)
    y = np.linspace(min_S2, max_S2, resolution)
    X,Y = np.meshgrid(x, y)

    U = S1_ODE(X,Y)
    V = S2_ODE(X,Y)


    F1 = ((S1_ODE(X,Y)<tolerance) & (S1_ODE(X,Y)>-tolerance))
    F2 = ((S2_ODE(X,Y)<tolerance) & (S2_ODE(X,Y)>-tolerance))


    fig, ax = plt.subplots()
    ax.streamplot(X, Y, U, V, color='gray')
    ax.contour(X,Y,F1, colors='orange')
    ax.contour(X,Y,F2, colors='purple')
    ax.set_aspect('equal', 'box')
    fig.suptitle('Flow and Nullclines',fontweight="bold")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.show()

gen_null_flow(-2, 2, -2, 2, 4000, 0.001)

Fixed Points in 2-Dimensions:

In [ ]:
#####################################################################################

# CODE FOR FIXED POINTS

# WORKFLOW:
# 1. Take in a matrix of proposed fixed points.
# 2. For each proposed fixed point, numerically solve for the root.
# 3. Return the numerical solutions as a corresponding matrix.
# 4. For any numerical solutions beneath a certain threshold, set them to zero.

def equil(state):
    return [S1_ODE(state[0],state[1]), S2_ODE(state[0],state[1])]

# Here, you need to provide the numerical solver with some guesses. These guesses could have been found qualitatively
# (by looking at the nullclines). Any guesses that converge on the same solution will just return that solution.  
guesses = np.array([[0, 0.5], [0, -0.5], [-1, -1]])

def find_eqs(guesses):
    sols = np.zeros((len(guesses),2))
    for i in range(len(guesses)):
        sols[i] = opt.fsolve(equil, guesses[i])
    sols[abs(sols) < 0.000001] = 0
    return sols

roots = find_eqs(guesses)

print(roots)

Jacobian Matrix in 2-Dimensions:

In [ ]:
f1 = lambda x,y: x**3 + y**2 - 0.25
f2 = lambda x,y: -y**3 + x
F = [f1,f2]

def Jacobian(system):
    x,y = sympy.symbols('x y')
    f1 = system[0]
    f2 = system[1]
    J = sympy.Matrix([[sympy.diff(f1(x,y),x),sympy.diff(f1(x,y),y)],[sympy.diff(f2(x,y),x),sympy.diff(f2(x,y),y)]])
    return(J)

J = Jacobian(F)

Stability in 2-Dimensions:

In [ ]:
f1 = lambda x,y: x**3 + y**2 - 0.25
f2 = lambda x,y: -y**3 + x
F = [f1,f2]

def find_eqs(guesses):
    sols = np.zeros((len(guesses),2))
    for i in range(len(guesses)):
        sols[i] = opt.fsolve(equil, guesses[i])
    sols[abs(sols) < 0.000001] = 0
    return sols

def equil(state):
    return [S1_ODE(state[0],state[1]), S2_ODE(state[0],state[1])]

def EP_Stability(J, guesses):
    roots = find_eqs(guesses)
    eigenvalues = list(J.eigenvals())
    stabilities = list()
    for p in roots:
        x = p[0]
        y = p[1]
        output = list()
        for e in eigenvalues:
            sub = sympy.re(e.subs({'x':x,'y':y}))
            output.append(sub)
        if all(i > 0 for i in output):
            print("The equilibrium point {} is unstable.".format(p))
            stabilities.append(0)
        elif all(i < 0 for i in output):
            print("The equilibrium point {} is stable.".format(p))
            stabilities.append(1)
        else:
            print("The equilibrium point {} is a saddle.".format(p))
            stabilities.append(2)
    return stabilities

guesses = np.array([[0, 0.5], [0, -0.5], [-1, -1]])
EP_Stability(Jacobian(F), guesses)

Phase Portrait in 2-Dimensions:

In [ ]:
#####################################################################################

# CODE FOR PHASE PORTRAIT (NO STABILITY)

# WORKFLOW:
# 1. Take in a matrix of confirmed fixed points.
# 2. Plot the Nullclines and Flow with the fixed points.

# (THIS IS PROOF OF CONCEPT. WE CAN MAKE THIS BETTER AND BREAK UP THE FUNCTIONS DIFFERENTLY.)

def gen_phase(min_S1, max_S1, min_S2, max_S2, resolution, tolerance, roots):

    x = np.linspace(min_S1, max_S1, resolution)
    y = np.linspace(min_S2, max_S2, resolution)
    X,Y = np.meshgrid(x, y)

    U = S1_ODE(X,Y)
    V = S2_ODE(X,Y)

    F1 = ((S1_ODE(X,Y)<tolerance) & (S1_ODE(X,Y)>-tolerance))
    F2 = ((S2_ODE(X,Y)<tolerance) & (S2_ODE(X,Y)>-tolerance))

    fig, ax = plt.subplots()
    ax.streamplot(X, Y, U, V, color='gray')
    ax.contour(X,Y,F1, colors='orange')
    ax.contour(X,Y,F2, colors='purple')
    ax.set_aspect('equal', 'box')
    for i in range(len(roots)):
        stab = EP_Stability(Jacobian(F), [roots[i]])
        if stab[0] == 0:
            ax.plot(roots[i][0], roots[i][1], 'ro')
        elif stab[0] == 1:
            ax.plot(roots[i][0], roots[i][1], 'bo')
        elif stab[0] == 2:
            ax.plot(roots[i][0], roots[i][1], 'go')
        else:
            ax.plot(roots[i][0], roots[i][1], 'black')
    fig.suptitle('Flow and Nullclines',fontweight="bold")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.show()

guesses = np.array([[0, 0.5], [0, -0.5], [-1, -1]])
roots = find_eqs(guesses)
gen_phase(-2, 2, -2, 2, 4000, 0.001, roots)